In [1]:
import multiprocessing
multiprocessing.set_start_method('spawn', force=True)
import random
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from tensorflow import keras
import matplotlib.pyplot as plt
import joblib
from multiprocessing import Pool
!pip install anthropic
import anthropic
import os
os.environ["ANTHROPIC_API_KEY"] = "your-actual-api-key-here"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [2]:
def ask_claude(prompt):
    client = anthropic.Client(os.environ["ANTHROPIC_API_KEY"])
    try:
        response = client.completion(
            prompt=f"{anthropic.HUMAN_PROMPT} {prompt}{anthropic.AI_PROMPT}",
            model="claude-2.0",
            max_tokens_to_sample=300,
            temperature=0.7,
        )
        return response.completion
    except Exception as e:
        print(f"Error querying Claude: {e}")
        return None

In [3]:
# Tesla Model Specifications (approximate values)
tesla_models = {
    'Model 3 RWD': {'weight': 1752, 'motor_power': 202, 'battery_capacity': 57.5, 'drag_coefficient': 0.23, 'wear_rate': 1.3},
    'Model 3 Long Range': {'weight': 1844, 'motor_power': 324, 'battery_capacity': 82, 'drag_coefficient': 0.23, 'wear_rate': 1.3},
    'Model 3 Performance': {'weight': 1847, 'motor_power': 413, 'battery_capacity': 82, 'drag_coefficient': 0.23, 'wear_rate': 1.7},
    'Model S': {'weight': 2069, 'motor_power': 500, 'battery_capacity': 100, 'drag_coefficient': 0.208, 'wear_rate': 1.4},
    'Model S Plaid': {'weight': 2162, 'motor_power': 760, 'battery_capacity': 100, 'drag_coefficient': 0.208, 'wear_rate': 1.7},
    'Model X': {'weight': 2352, 'motor_power': 500, 'battery_capacity': 100, 'drag_coefficient': 0.24, 'wear_rate': 1.4},
    'Model X Plaid': {'weight': 2455, 'motor_power': 760, 'battery_capacity': 100, 'drag_coefficient': 0.24, 'wear_rate': 1.7},
    'Model Y RWD': {'weight': 1909, 'motor_power': 220, 'battery_capacity': 60, 'drag_coefficient': 0.23, 'wear_rate': 1.3},
    'Model Y Long Range': {'weight': 2003, 'motor_power': 324, 'battery_capacity': 75, 'drag_coefficient': 0.23, 'wear_rate': 1.3},
    'Model Y Performance': {'weight': 2033, 'motor_power': 384, 'battery_capacity': 75, 'drag_coefficient': 0.23, 'wear_rate': 1.7},
    'Cybertruck Single Motor': {'weight': 2721, 'motor_power': 300, 'battery_capacity': 100, 'drag_coefficient': 0.3, 'wear_rate': 1.4},
    'Cybertruck Dual Motor': {'weight': 2948, 'motor_power': 500, 'battery_capacity': 120, 'drag_coefficient': 0.3, 'wear_rate': 1.4},
    'Cybertruck Tri Motor': {'weight': 3175, 'motor_power': 700, 'battery_capacity': 150, 'drag_coefficient': 0.3, 'wear_rate': 1.7}
}

# Tire specifications (approximate values)
tire_specs = {
    'Model 3 RWD': {'size': '18"', 'tread_wear': 400, 'load_index': 99, 'speed_rating': 'W', 'starting_tread': 8},
    'Model 3 Long Range': {'size': '18"', 'tread_wear': 400, 'load_index': 99, 'speed_rating': 'W', 'starting_tread': 8},
    'Model 3 Performance': {'size': '20"', 'tread_wear': 300, 'load_index': 101, 'speed_rating': 'Y', 'starting_tread': 8},
    'Model S': {'size': '19"', 'tread_wear': 300, 'load_index': 105, 'speed_rating': 'W', 'starting_tread': 8},
    'Model S Plaid': {'size': '21"', 'tread_wear': 280, 'load_index': 107, 'speed_rating': 'Y', 'starting_tread': 8},
    'Model X': {'size': '20"', 'tread_wear': 300, 'load_index': 108, 'speed_rating': 'W', 'starting_tread': 8},
    'Model X Plaid': {'size': '22"', 'tread_wear': 285, 'load_index': 110, 'speed_rating': 'Y', 'starting_tread': 8},
    'Model Y RWD': {'size': '19"', 'tread_wear': 350, 'load_index': 101, 'speed_rating': 'W', 'starting_tread': 8},
    'Model Y Long Range': {'size': '19"', 'tread_wear': 350, 'load_index': 101, 'speed_rating': 'W', 'starting_tread': 8},
    'Model Y Performance': {'size': '21"', 'tread_wear': 300, 'load_index': 103, 'speed_rating': 'Y', 'starting_tread': 8},
    'Cybertruck Single Motor': {'size': '35"', 'tread_wear': 500, 'load_index': 121, 'speed_rating': 'T', 'starting_tread': 10},
    'Cybertruck Dual Motor': {'size': '35"', 'tread_wear': 500, 'load_index': 121, 'speed_rating': 'T', 'starting_tread': 10},
    'Cybertruck Tri Motor': {'size': '35"', 'tread_wear': 500, 'load_index': 121, 'speed_rating': 'T', 'starting_tread': 10}
}

In [4]:
def calculate_wear_factor(vehicle_model, trip_distance, speed, acceleration, temperature, road_condition):
    """
    Calculate the wear factor for a given trip.

    This function considers various factors that affect tire wear, including
    vehicle specifications, trip characteristics, and environmental conditions.
    """
    try:
        model_specs = tesla_models[vehicle_model]
        tire_spec = tire_specs[vehicle_model]

        base_wear = (trip_distance / 6500) * (model_specs['wear_rate'] / 32)
        weight_factor = (model_specs['weight'] / 1800) ** 0.5
        speed_factor = (speed / 65) ** 1.5
        power_factor = (model_specs['motor_power'] / 300) ** 0.3
        accel_factor = 1 + abs(acceleration) / 5
        temp_factor = 1 + (temperature - 20) / 100 if temperature > 20 else 1
        road_factors = {'dry': 1, 'wet': 1.2, 'icy': 1.5}
        road_factor = road_factors[road_condition]
        tread_factor = 300 / tire_spec['tread_wear']
        size_factor = float(tire_spec['size'].rstrip('"')) / 18

        total_wear = base_wear * weight_factor * speed_factor * power_factor * accel_factor * temp_factor * road_factor * tread_factor * size_factor

        return total_wear
    except KeyError as e:
        print(f"Invalid key in vehicle or tire specifications: {e}")
        return None
    except Exception as e:
        print(f"Error in wear factor calculation: {e}")
        return None

def simulate_trip(vehicle_model, trip_distance, speed, acceleration, temperature, road_condition, current_tread):
    """Simulate a single trip and calculate remaining tread depth for each tire."""
    wear = calculate_wear_factor(vehicle_model, trip_distance, speed, acceleration, temperature, road_condition)
    if wear is None:
        return None

    wear *= random.uniform(0.9, 1.1)
    remaining_tread = [max(0, current_tread - wear * random.uniform(0.95, 1.05)) for _ in range(4)]
    return remaining_tread

In [5]:
def simulate_car(car_params):
    """Simulate trips for a single car until tire replacement is needed."""
    vehicle_model, max_steps = car_params
    starting_tread = tire_specs[vehicle_model]['starting_tread']
    current_tread = starting_tread
    total_distance = 0
    results = []
    replaced_tire = None

    for step in range(max_steps):
        trip_distance = random.uniform(10, 500)
        total_distance += trip_distance
        speed = random.uniform(30, 75)
        acceleration = random.uniform(-3, 3)
        temperature = random.uniform(0, 100)
        road_condition = random.choice(['dry', 'wet', 'icy'])

        remaining_tread = simulate_trip(vehicle_model, trip_distance, speed, acceleration, temperature, road_condition, current_tread)
        if remaining_tread is None:
            continue

        current_tread = min(remaining_tread)

        results.append({
            'step': step,
            'vehicle_model': vehicle_model,
            'total_distance': total_distance,
            'current_tread': current_tread,
            'tread_fl': remaining_tread[0],
            'tread_fr': remaining_tread[1],
            'tread_rl': remaining_tread[2],
            'tread_rr': remaining_tread[3],
        })

        if current_tread <= 4:
            replaced_tire = {
                'vehicle_model': vehicle_model,
                'total_distance': total_distance,
                'final_tread': current_tread,
                'steps': step
            }
            break

    return results, replaced_tire

def run_simulation(num_cars=100, max_steps=100):
    """Run the simulation for multiple cars in parallel."""
    car_params = [(random.choice(list(tesla_models.keys())), max_steps) for _ in range(num_cars)]

    with Pool() as pool:
        results = pool.map(simulate_car, car_params)

    all_results = []
    replaced_tires = []
    for car_results, replaced_tire in results:
        all_results.extend(car_results)
        if replaced_tire:
            replaced_tires.append(replaced_tire)

    return pd.DataFrame(all_results), replaced_tires

In [6]:
def prepare_data(results):
    """Prepare data for machine learning models."""
    # Reshape the data to have one row per tire
    tire_data = []
    for _, row in results.iterrows():
        for position in ['fl', 'fr', 'rl', 'rr']:
            tire_data.append({
                'total_distance': row['total_distance'],
                'vehicle_model': row['vehicle_model'],
                'tire_position': position,
                'tread_depth': row[f'tread_{position}']
            })

    df = pd.DataFrame(tire_data)

    # Prepare features
    X = df[['total_distance', 'vehicle_model', 'tire_position']]
    X = pd.get_dummies(X, columns=['vehicle_model', 'tire_position'])
    X = X.apply(pd.to_numeric, errors='coerce')
    X.fillna(0, inplace=True)
    X = X.astype(np.float32)

    # Prepare target
    y = df['tread_depth']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, y, scaler

def train_and_evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Train and evaluate a single model."""
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    print(f"{model_name} MSE: {mse}")
    print(f"{model_name} R^2: {r2}")

    plot_model_accuracy(y_test, predictions, model_name)

    return predictions

def train_ml_models(results):
    """Train and evaluate multiple machine learning models."""
    X_scaled, y, scaler = prepare_data(results)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Random Forest
    rf_model = RandomForestRegressor(random_state=42)
    rf_predictions = train_and_evaluate_model(rf_model, X_train, X_test, y_train, y_test, "Random Forest")

    # XGBoost
    xgb_model = xgb.XGBRegressor(random_state=42)
    xgb_predictions = train_and_evaluate_model(xgb_model, X_train, X_test, y_train, y_test, "XGBoost")

    # Neural Network
    nn_model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1)
    ])
    nn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    nn_predictions = nn_model.predict(X_test).flatten()

    print(f"Neural Network MSE:", mean_squared_error(y_test, nn_predictions))
    print(f"Neural Network R^2:", r2_score(y_test, nn_predictions))
    plot_model_accuracy(y_test, nn_predictions, "Neural Network")

    # Ensemble
    ensemble_predictions = (rf_predictions + xgb_predictions + nn_predictions) / 3
    print(f"Ensemble MSE:", mean_squared_error(y_test, ensemble_predictions))
    print(f"Ensemble R^2:", r2_score(y_test, ensemble_predictions))
    plot_model_accuracy(y_test, ensemble_predictions, "Ensemble")

    # Save models
    joblib.dump(rf_model, 'rf_model.joblib')
    joblib.dump(xgb_model, 'xgb_model.joblib')
    nn_model.save('nn_model.h5')
    joblib.dump(scaler, 'scaler.joblib')

In [ ]:
def plot_model_accuracy(y_test, predictions, model_name):
    """Plot model accuracy and residuals."""
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.scatter(y_test, predictions, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Actual Tread Depth (32nds of an inch)')
    plt.ylabel('Predicted Tread Depth (32nds of an inch)')
    plt.title(f'{model_name}: Actual vs Predicted')

    plt.subplot(1, 2, 2)
    residuals = y_test - predictions
    plt.scatter(predictions, residuals, alpha=0.5)
    plt.plot([predictions.min(), predictions.max()], [0, 0], 'r--', lw=2)
    plt.xlabel('Predicted Tread Depth (32nds of an inch)')
    plt.ylabel('Residuals')
    plt.title(f'{model_name}: Residuals')

    plt.tight_layout()
    plt.show()

def analyze_replaced_tires(replaced_tires):
    for tire in replaced_tires:
        prompt = f"""
        A {tire['vehicle_model']} needed its tires replaced after driving {tire['total_distance']:.2f} miles.
        The final tread depth was {tire['final_tread']:.2f}/32 inches, and it took {tire['steps']} simulation steps to reach this point.
        Based on this information and your knowledge of factors affecting tire wear, what are the likely causes of this tire wear?
        Please provide a detailed explanation.
        """

        explanation = ask_claude(prompt)

        print(f"\nAnalysis for {tire['vehicle_model']} tire replacement:")
        print(explanation)

def get_tire_longevity_tips(simulation_results):
    average_distance = simulation_results['total_distance'].mean()
    prompt = f"""
    Based on our simulation, the average distance traveled before tire replacement was {average_distance:.2f} miles.
    What are some tips for improving tire longevity in electric vehicles like Teslas?
    Please provide 3-5 specific recommendations.
    """
    tips = ask_claude(prompt)
    print("\nTips for improving tire longevity:")
    print(tips)


def main():
    print("Tesla Tire Wear Prediction Model with Claude AI")
    try:
        simulation_results, replaced_tires = run_simulation(num_cars=100, max_steps=100)
        train_ml_models(simulation_results)

        print("\nAnalyzing replaced tires...")
        analyze_replaced_tires(replaced_tires)

        print("Simulation complete, ML models trained, and replaced tires analyzed by Claude.")
    except Exception as e:
        print(f"An error occurred during simulation or training: {e}")

if __name__ == "__main__":
    main()

Tesla Tire Wear Prediction Model with Claude AI
